In [4]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta

os.chdir("/Users/wanghanci/Desktop/Hanci/Spring 2018/DSO570 The Analytics Edges_Data Models and Effective Decisions/Project/Data/")

StudentSelection = pd.read_excel("Student_Course_Selection_1516.xlsx")
CourseEnrollment = pd.read_excel("Marshall_Course_Enrollment_1516_1617.xlsx")

### We focus on 2015 fall semester at this stage

Student_20153 = StudentSelection[StudentSelection.Term == 20153]
Course_20153 = CourseEnrollment[CourseEnrollment.Term == 20153]

### Merge two dataser by "Section"
Student_Course_20153 = pd.merge(left = Student_20153, right = Course_20153, how = "left", on = "Section")

### Filter out records without start time and end time
Student_Course_20153 = Student_Course_20153.dropna(subset = ["First Begin Time", "First End Time"])

### convert start time and end time to seconds
start = []
end = []
for index,row in Student_Course_20153.iterrows():
    
    startSec = (row["First Begin Time"].hour * 3600 +
                row["First Begin Time"].minute * 60 + 
                row["First Begin Time"].second * 1)
    start.append(startSec)
    
    endSec =  (row["First End Time"].hour * 3600 +
               row["First End Time"].minute * 60 + 
               row["First End Time"].second * 1)
    end.append(endSec)
    
Student_Course_20153["start"] = start
Student_Course_20153["end"] = end

SC20153 = Student_Course_20153[["Randomized Unique Identifier",
                                "Section",
                                "First Days",
                                "start",
                                "end"]]


### Create a dictionary to store the data
sc = {}

for index, row in SC20153.iterrows():
    id = row["Randomized Unique Identifier"]
    section = row["Section"]
    days = row["First Days"]
    start = row["start"]
    end = row["end"]
    for day in "MTWHF":
        if day in days:
            key = (id, section, day)
            if key not in sc:
                time = {}
                time["start"] = start
                time["end"] = end
                sc[key] = time


### convert the dictionary to dataframe
scdf = pd.DataFrame.from_dict(sc, orient = "index")

scdf["index"] = scdf.index
scdf.index = range(1,38109)

id = []
section = [] 
day = []

for index, row in scdf.iterrows():
    id.append(row["index"][0])
    section.append(row["index"][1])    
    day.append(row["index"][2])    
        
scdf["id"] = id
scdf["section"] = section
scdf["day"] = day    

scdf = scdf[["id", "section", "day", "start", "end"]]


### Reolace "M","T","W","H","F" with 1,2,3,4,5
for index, day in enumerate(["M","T", "W", "H", "F"]):
    scdf["day"] = scdf["day"].replace(day, index + 1)


### integrete the start and end into one column in order to compute the gap
scdf = pd.melt(scdf, id_vars = ["id", "section", "day"], value_vars = ["start", "end"],
              var_name = "start_end", value_name = "time")


### group by id and sort by day and time
scdf = scdf.groupby(["id"]).apply(lambda x: x.sort_values(["day","time"])).reset_index(drop=True) 

temp = scdf.copy()
scdf

### Calculate the interval for each student, each day
scdf['interval'] = scdf.groupby(["id", "day"])['time'].diff()

### keep the rows having class interval
final = scdf[scdf["start_end"] == "start"]

### group by id and check if there is a interval larger than 600 seconds
final.groupby(["id"]).max()

### Caculate the percentage of students who have "gap"
final["gap"] = final["interval"] > 600
sum(final["gap"])/len(final)

### Conclusion: 17% of the students suffered from "gaps" on 2015 Fall

/Users/pratiksha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.17046289493019839